In [ ]:
# =========================================================
# [Cell 1] 환경 설정 및 데이터 준비 (SAM 기반)
# =========================================================
!pip install roboflow torch torchvision torchaudio opencv-python numpy tqdm pillow matplotlib albumentations timm segment-anything

import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from roboflow import Roboflow
from tqdm import tqdm
from segment_anything import sam_model_registry

# SAM은 1024x1024 입력
IMG_SIZE = 1024 
BATCH_SIZE = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"🔥 Device: {device}, Resolution: {IMG_SIZE}x{IMG_SIZE}")

# 1. SAM 가중치 다운로드 (ViT-Base)
if not os.path.exists("sam_vit_b_01ec64.pth"):
    print("⬇️ Downloading SAM weights...")
    !wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth

# 2. 데이터셋 다운로드
rf = Roboflow(api_key="HG9M6YJZpcCUgAQaKO9v")
project = rf.workspace("arakon").project("detection-base-hqaeg")
version = project.version(6)
dataset = version.download("yolov8")

# 3. 데이터셋 클래스 (1024px 리사이징)
def yolo_collate_fn(batch):
    imgs = torch.stack([b[0] for b in batch], 0)
    targets = [b[1] for b in batch]
    return imgs, targets

class YoloDataset(Dataset):
    def __init__(self, root, is_train=True):
        self.img_dir = os.path.join(root, "images")
        self.label_dir = os.path.join(root, "labels")
        self.files = sorted([x for x in os.listdir(self.img_dir) if x.endswith(('.jpg', '.png', '.jpeg'))])
        self.img_size = IMG_SIZE

    def __len__(self): return len(self.files)

    def __getitem__(self, idx):
        name = self.files[idx]
        img_path = os.path.join(self.img_dir, name)
        img = cv2.imread(img_path)
        
        # SAM은 RGB 입력을 받습니다.
        img = cv2.resize(img, (self.img_size, self.img_size))
        img_tensor = torch.from_numpy(img[:, :, ::-1].copy()).permute(2, 0, 1).float() 
        # SAM Preprocessing: (pixel - mean) / std (여기선 단순 0-1 정규화 후 모델 내부 처리 믿음)
        # 하지만 SAM은 원래 0-255 입력을 기대하는 경우가 많으나, 여기선 표준적인 0-1 정규화 사용
        # (Segment Anything 라이브러리 내부 preprocess가 처리해주지만, 여기선 Custom Head라 직접 함)
        img_tensor = img_tensor / 255.0 

        label_path = os.path.join(self.label_dir, name.rsplit('.', 1)[0] + ".txt")
        boxes = []
        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                for line in f.readlines():
                    vals = list(map(float, line.strip().split()))
                    if len(vals) == 5:
                        boxes.append(vals) # cls, cx, cy, w, h
        
        target = torch.tensor(boxes, dtype=torch.float32) if len(boxes) > 0 else torch.zeros((0, 5))
        return img_tensor, target

train_loader = DataLoader(YoloDataset(dataset.location + "/train"), batch_size=BATCH_SIZE, shuffle=True, collate_fn=yolo_collate_fn, num_workers=2)
val_loader = DataLoader(YoloDataset(dataset.location + "/valid"), batch_size=BATCH_SIZE, shuffle=False, collate_fn=yolo_collate_fn, num_workers=2)
test_loader = DataLoader(YoloDataset(dataset.location + "/test"), batch_size=BATCH_SIZE, shuffle=False, collate_fn=yolo_collate_fn, num_workers=2)

print("✅ Data Ready.")

In [ ]:
# =========================================================
# [Cell 2] SAM Backbone + Adapter + Detection Head
# =========================================================

class ConvNormAct(nn.Module):
    def __init__(self, in_c, out_c, k=3, s=1, p=1):
        super().__init__()
        self.conv = nn.Conv2d(in_c, out_c, k, s, p, bias=False)
        self.bn = nn.BatchNorm2d(out_c)
        self.act = nn.SiLU()
    def forward(self, x): return self.act(self.bn(self.conv(x)))

class SAMDetector(nn.Module):
    def __init__(self, checkpoint_path="sam_vit_b_01ec64.pth", num_classes=3):
        super().__init__()
        
        # 1. Load SAM (ViT-Base)
        print("🔄 Loading SAM Backbone...")
        self.sam = sam_model_registry["vit_b"](checkpoint=checkpoint_path)
        
        # [핵심] Backbone Freeze (학습 안 함 -> 메모리 절약, 특징 보존)
        for param in self.sam.image_encoder.parameters():
            param.requires_grad = False
            
        # SAM ViT-B Output channel = 256
        self.sam_out_dim = 256
        
        # 2. Adapter (Neck / FPN)
        # SAM 출력은 (B, 256, 64, 64) 하나뿐임 (1024px 기준, stride 16)
        # 이를 통해 P3(stride 8), P4(stride 16), P5(stride 32)를 만들어야 함.
        
        # P4: 그대로 사용하거나 정제
        self.p4_conv = ConvNormAct(256, 256)
        
        # P3: 2배 Upsample
        self.p3_up = nn.Upsample(scale_factor=2, mode='nearest')
        self.p3_conv = ConvNormAct(256, 256)
        
        # P5: 2배 Downsample
        self.p5_down = ConvNormAct(256, 256, k=3, s=2, p=1)
        
        # 3. Detection Head (YOLO Style)
        # 각 레벨(P3, P4, P5)마다 예측 수행
        self.heads = nn.ModuleList([
            nn.Conv2d(256, num_classes + 5, 1) for _ in range(3) # 5 = 1(obj) + 4(box)
        ])
        
    def forward(self, x):
        # 1. Backbone (Frozen)
        # SAM은 입력 이미지를 (1024, 1024)로 기대하며 내부적으로 정규화함.
        # 여기선 간단히 0-1 텐서를 SAM preprocess에 맞춰 스케일링하는 게 좋지만,
        # 학습 가능한 Adapter가 보정해주므로 그대로 넣어도 무방.
        with torch.no_grad():
            # (B, 3, 1024, 1024) -> (B, 256, 64, 64)
            features = self.sam.image_encoder(x)
            
        # 2. Neck (Generate Multi-scale Features)
        p4 = self.p4_conv(features)          # (B, 256, 64, 64) - Stride 16
        p3 = self.p3_conv(self.p3_up(p4))    # (B, 256, 128, 128) - Stride 8
        p5 = self.p5_down(p4)                # (B, 256, 32, 32) - Stride 32
        
        # 3. Head
        outputs = []
        for i, feat in enumerate([p3, p4, p5]):
            out = self.heads[i](feat) # (B, C+5, H, W)
            
            # Split output
            # cls: 0~C, obj: C, box: C+1~C+5
            cls_p = out[:, :3, ...]
            obj_p = out[:, 3:4, ...]
            box_p = out[:, 4:, ...]
            outputs.append((cls_p, obj_p, box_p))
            
        return outputs

print("✅ SAM Detector Defined.")

In [ ]:
# =========================================================
# [Cell 3] Loss Function & Utils
# =========================================================

def box_iou(box1, box2):
    lt = torch.max(box1[:, None, :2], box2[:, :2])
    rb = torch.min(box1[:, None, 2:], box2[:, 2:])
    inter = (rb - lt).clamp(min=0).prod(2)
    area1 = (box1[:, 2]-box1[:, 0]) * (box1[:, 3]-box1[:, 1])
    area2 = (box2[:, 2]-box2[:, 0]) * (box2[:, 3]-box2[:, 1])
    return inter / (area1[:, None] + area2 - inter + 1e-6)

def select_candidates(xy_centers, gt_bboxes):
    bs, n_boxes, _ = gt_bboxes.shape
    lt, rb = gt_bboxes.view(-1, 1, 4).chunk(2, 2)
    bbox_deltas = torch.cat((xy_centers[None] - lt, rb - xy_centers[None]), dim=2).view(bs, n_boxes, -1, 4)
    return bbox_deltas.min(3)[0] > 1e-9

class ComputeLoss(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()
        self.bce = nn.BCEWithLogitsLoss(reduction='none')
        self.num_classes = num_classes

    def forward(self, preds, batch_targets, img_size):
        # preds: list of (cls, obj, box)
        loss_total = 0.0
        strides = [8, 16, 32]
        
        batch_size = preds[0][0].shape[0]
        
        # 앵커 생성 및 예측값 디코딩
        for i, (cls_p, obj_p, box_p) in enumerate(preds):
            stride = strides[i]
            B, _, H, W = cls_p.shape
            
            grid_y, grid_x = torch.meshgrid(torch.arange(H), torch.arange(W), indexing='ij')
            grid = torch.stack((grid_x, grid_y), 2).to(cls_p.device).float()
            
            # Box Decoding
            b_sig = box_p.permute(0, 2, 3, 1).sigmoid() # (B, H, W, 4)
            pred_cx = (b_sig[..., 0] * 2.0 - 0.5 + grid[..., 0]) * stride
            pred_cy = (b_sig[..., 1] * 2.0 - 0.5 + grid[..., 1]) * stride
            pred_w = (b_sig[..., 2] * 2.0) ** 2 * stride
            pred_h = (b_sig[..., 3] * 2.0) ** 2 * stride
            
            pred_xyxy = torch.stack([pred_cx - pred_w/2, pred_cy - pred_h/2, 
                                     pred_cx + pred_w/2, pred_cy + pred_h/2], dim=-1)
            
            # --- Target Matching (Simplified Anchor-free) ---
            # 각 GT 박스 안에 중심이 들어오는 앵커를 Positive로 간주
            anchors_xy = (grid + 0.5) * stride # (H, W, 2)
            anchors_flat = anchors_xy.view(-1, 2)
            
            obj_target = torch.zeros(B, H, W, 1, device=cls_p.device)
            cls_target = torch.zeros(B, H, W, self.num_classes, device=cls_p.device)
            
            loss_box_layer = 0.0
            num_pos_layer = 0
            
            for b in range(B):
                gt = batch_targets[b]
                if len(gt) == 0: continue
                
                # GT: [cls, cx, cy, w, h] (norm) -> xyxy (pixel)
                gt_cls = gt[:, 0].long()
                cx, cy, w, h = gt[:, 1]*img_size, gt[:, 2]*img_size, gt[:, 3]*img_size, gt[:, 4]*img_size
                x1, y1, x2, y2 = cx - w/2, cy - h/2, cx + w/2, cy + h/2
                gt_xyxy = torch.stack([x1, y1, x2, y2], dim=1).to(cls_p.device)
                
                # Check anchors in GT
                # anchors_flat: (N_anc, 2)
                # gt_xyxy: (N_gt, 4)
                # mask: (N_gt, N_anc)
                lt = gt_xyxy[:, None, :2]
                rb = gt_xyxy[:, None, 2:]
                # (N_gt, N_anc, 2)
                deltas = torch.cat((anchors_flat[None] - lt, rb - anchors_flat[None]), dim=2)
                is_in_gt = deltas.min(dim=2)[0] > 0 # (N_gt, N_anc)
                
                # 각 앵커별로 매칭된 GT 찾기 (하나라도 들어있으면 OK)
                # 여기선 간단히 '마지막 GT'에 매칭 (복잡한 할당 로직 대신)
                matched_gt_idx = is_in_gt.max(dim=0)[1] # (N_anc,)
                has_match = is_in_gt.max(dim=0)[0] # (N_anc,)
                
                if has_match.sum() > 0:
                    pos_mask = has_match.view(H, W)
                    
                    # Obj Target
                    obj_target[b, pos_mask] = 1.0
                    
                    # Cls Target
                    matched_labels = gt_cls[matched_gt_idx[has_match]]
                    cls_target[b, pos_mask, matched_labels] = 1.0
                    
                    # Box Loss (CIoU)
                    pred_box_pos = pred_xyxy[b, pos_mask]
                    gt_box_pos = gt_xyxy[matched_gt_idx[has_match]]
                    iou = box_iou(pred_box_pos, gt_box_pos).diag()
                    loss_box_layer += (1.0 - iou).sum()
                    num_pos_layer += has_match.sum()

            # Loss Accumulation
            # Objectness (Focal style)
            obj_pred_flat = obj_p.permute(0, 2, 3, 1).reshape(-1, 1)
            obj_tgt_flat = obj_target.reshape(-1, 1)
            loss_obj = F.binary_cross_entropy_with_logits(obj_pred_flat, obj_tgt_flat, reduction='sum')
            
            # Cls
            cls_pred_flat = cls_p.permute(0, 2, 3, 1).reshape(-1, self.num_classes)
            cls_tgt_flat = cls_target.reshape(-1, self.num_classes)
            loss_cls = F.binary_cross_entropy_with_logits(cls_pred_flat, cls_tgt_flat, reduction='sum')
            
            # Normalize
            num_pos = max(1.0, num_pos_layer)
            loss_total += (loss_obj + loss_cls + loss_box_layer * 5.0) / num_pos / B

        return loss_total

criterion = ComputeLoss()

In [ ]:
# =========================================================
# [Cell 4] SAM-Adapter Training Loop
# =========================================================

# 모델 초기화
model = SAMDetector(num_classes=3).to(device)

# 학습 가능한 파라미터만 Optimizer에 전달 (중요!)
trainable_params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(trainable_params, lr=1e-4, weight_decay=0.01)
scaler = torch.amp.GradScaler('cuda')

EPOCHS = 15
print(f"🚀 Start Training SAM-Adapter (1024x1024)...")
print(f"📌 Trainable Params: {len(trainable_params)}")

for epoch in range(EPOCHS):
    model.train()
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    epoch_loss = 0.0
    
    for i, (imgs, targets) in enumerate(loop):
        imgs = imgs.to(device)
        
        with torch.amp.autocast('cuda'):
            preds = model(imgs)
            loss = criterion(preds, targets, IMG_SIZE)
            
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        epoch_loss += loss.item()
        loop.set_postfix(loss=f"{loss.item():.4f}")
        
    print(f"Epoch {epoch+1} Avg Loss: {epoch_loss / len(train_loader):.4f}")
    
    # Checkpoint
    torch.save(model.state_dict(), "sam_adapter_best.pt")

print("🏁 Training Finished!")